In [2]:
import numpy as np
import importlib
import Layout
import Greedy


import copy 

#Reset libreria
#importlib.reload(px)
importlib.reload(Greedy)
importlib.reload(Layout)

def showtime2(path):
    layout=px.read(path)
    
    
    #print ("Estado inicial: ")
    #for s in layout.stacks:
    #    print(s)
    #print()
    Greedy.lazy_greedy(layout)
    seleccionada = SSRC(layout)
 
    while seleccionada != -1:
        
        if verbose:
            for s in layout.stacks: print(s)
            print("\n")
        
        #Se calcula cuantos contenedores deben retirarse
        columna=copy.deepcopy(layout.stacks[seleccionada])
        nb_unsorted_elements = px.remove_until(columna)

        #Se calculan los ranks
        rank = Greedy.get_ranks(layout.stacks[seleccionada])
        if verbose: print("selected:",layout.stacks[seleccionada])

        #Se retiran los elementos de la columna
        SDpp(layout, seleccionada, rank, nb_unsorted_elements)
        
        if verbose:
            for s in layout.stacks: print(s)
            print("\n")

        #Se quitan los contenedores que se mantuvieron en su lugar para que no tire error la siguiente funcion
        for item in layout.stacks[seleccionada]:
            rank.pop(item, None)

        #Según yo el último parámetro implica que el método tratará de llenar el stack, sin embargo si pongo 100 se cae?
        SFpp(layout,seleccionada,rank, layout.H-len(layout.stacks[seleccionada]) )#nb_unsorted_elements)

        Greedy.lazy_greedy(layout)
        #Se vuelve a seleccionar una columna
        seleccionada = SSRC(layout)

    #print("SELECCIONADA:", seleccionada)
    #print(len(layout.moves),"\t",layout.unsorted_stacks)
    if verbose: 
        for s in layout.stacks: print(s)
    return len(layout.moves)
    #print()
    
def solve2(path):
        layout=px.read(path)
    
    
        #print ("Estado inicial: ")
        #for s in layout.stacks:
        #    print(s)
        #print()
        Greedy.lazy_greedy(layout)
        seleccionada = SSRC(layout)

        while seleccionada != -1:

            #Se calcula cuantos contenedores deben retirarse
            columna=copy.deepcopy(layout.stacks[seleccionada])
            nb_unsorted_elements = px.remove_until(columna)

            #Se calculan los ranks
            rank = Greedy.get_ranks(layout.stacks[seleccionada])
            #if verbose: print("selected:",layout.stacks[seleccionada])

            #Se retiran los elementos de la columna
            SDpp(layout, seleccionada, rank, nb_unsorted_elements)

            #if verbose:
            #   for s in layout.stacks: print(s)
            #   print("\n")

            #Se quitan los contenedores que se mantuvieron en su lugar para que no tire error la siguiente funcion
            for item in layout.stacks[seleccionada]:
                rank.pop(item, None)

            #Según yo el último parámetro implica que el método tratará de llenar el stack, sin embargo si pongo 100 se cae?
            SFpp(layout,seleccionada,rank, layout.H-len(layout.stacks[seleccionada]) )#nb_unsorted_elements)

            Greedy.lazy_greedy(layout)
            #Se vuelve a seleccionar una columna
            seleccionada = SSRC(layout)

        #print("SELECCIONADA:", seleccionada)
        print(len(layout.moves),"\t",layout.unsorted_stacks)

        #print()
    

In [3]:
#selector de columnas a desmantelar
def SSRC(layout):
    best_ev = 0; s_o=-1
    for i in range(len(layout.stacks)):
        if layout.is_sorted_stack(i): continue
        prom = sum(layout.stacks[i]) / len(layout.stacks[i]) 
        ev = 10000 - 100*len(layout.stacks[i]) - prom

        if ev > best_ev:
            best_ev = ev
            s_o = i
    return s_o

In [4]:
def SDpp(layout, s_o, rank, n_elements=100):
    capac = Greedy.capacity(layout,s_o) # espacio libre
    
    ss_o = layout.stacks[s_o]
    
    fixed_elements = np.maximum(len(ss_o)-n_elements,0)
    
    while len(layout.stacks[s_o]) > fixed_elements:
        top = Layout.gvalue(ss_o)
        slack = capac-len(ss_o) # holgura
        #print(rank[top])
        s_d = Layout.select_destination_stack (layout, s_o, max_pos=rank[top]+slack, rank=rank)
        pos = layout.H - len(layout.stacks[s_d])
        if rank[top] < pos - slack:  # rellenar stack s_d
            #c=layout.stacks[s_o].pop(-1)
            if verbose: 
                for s in layout.stacks: print(s)
                print(s_d,(pos - slack) - rank[top],s_o, rank)
            Greedy.fill_stack(layout, s_d, (pos - slack) - rank[top], s_o, rank)
            #layout.stacks[s_o].append(c)

        capac -= 1
        layout.move(s_o,s_d)

In [5]:
import sys
def SFpp(layout, s_d, rank, n):
    ub = Layout.gvalue(layout.stacks[s_d])
    cont = 0
    for r in rank:
        while True:
            (s, pos) = Greedy.search_highest(layout, r, ub, s_d)  
            #if verbose:
                #print("r,ub:",r,ub)
                #print(layout.stacks[s][pos])
                
            if layout.sorted_elements[s] > len(layout.stacks[s])+pos: break #element is sorted
            ub=layout.stacks[s][pos]-1
                      
            c=Greedy.force_move(layout, s, pos, s_d)
            
            
            if c == None:    
                if ub<r: 
                    for stack in layout.stacks:
                        print(stack)
                    print(str(ub+1)+ " del stack " +str(s)+ " no se puede volver a colocar en "+str(s_d)+"!!");
                    sys.exit()
            else: 
                cont+=1
                if cont == n: return
                if r==c: break
    while cont < n:
        Greedy.SF_move_d(layout, s_d)
        cont += 1

In [8]:
import Greedy as gr
import importlib
importlib.reload(Greedy)
importlib.reload(Layout)
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir("Instancias") if not isfile(join("Instancias", f))]

verbose = False
tot_moves=0
for f in onlyfiles:
    if f == 'BF' or f== '3-x' : continue
    #print(f)
    nmoves=0
    for i in range(1,41):
        filename = "Instancias/"+ f+ "/data"+f+"-"+str(i)+".dat"
        nmoves += showtime2(filename)
    tot_moves += nmoves
    print (f,nmoves/ 40.0);
print("prom:",tot_moves/(21.0*40.0))

10-10 160.975
10-6 124.775
3-3 12.9
3-4 13.75
3-5 13.925
3-6 14.9
3-7 16.325
3-8 16.925
4-4 23.475
4-5 25.65
4-6 26.775
4-7 30.15
5-10 54.55
5-4 35.05
5-5 37.075
5-6 41.45
5-7 42.25
5-8 48.8
5-9 51.9
6-10 73.65
6-6 54.425
prom: 43.79404761904762


![image](https://i.imgur.com/XsBW9YR.png)

In [7]:
import Pixie as px

verbose = False
f='3-5'
for i in range(1,41):
    filename = "Instancias/"+ f+ "/data"+f+"-"+str(i)+".dat"
    solve2(filename)
i=14


10 	 0
16 	 0
15 	 0
8 	 0
18 	 0
21 	 0
9 	 0
18 	 0
17 	 0
14 	 0
20 	 0
19 	 0
20 	 0
9 	 0
17 	 0
11 	 0
18 	 0
9 	 0
15 	 0
19 	 0
11 	 0
12 	 0
14 	 0
12 	 0
8 	 0
18 	 0
10 	 0
16 	 0
18 	 0
9 	 0
17 	 0
9 	 0
14 	 0
12 	 0
14 	 0
14 	 0
19 	 0
8 	 0
9 	 0
10 	 0


In [9]:
verbose = True
i=14
filename = "Instancias/"+ f+ "/data"+f+"-"+str(i)+".dat"
print(i,showtime2(filename))

FileNotFoundError: [Errno 2] No such file or directory: 'Instancias/BF/dataBF-14.dat'

In [101]:
verbose = False
f='3-5'
for i in range(1,41):
    filename = "Instancias/"+ f+ "/data"+f+"-"+str(i)+".dat"
    solve2(filename)

10 	 0
16 	 0
15 	 0
8 	 0
18 	 0
21 	 0
9 	 0
18 	 0
17 	 0
14 	 0
20 	 0
19 	 0
20 	 0
9 	 0
17 	 0
11 	 0
18 	 0
9 	 0
15 	 0
19 	 0
11 	 0
12 	 0
14 	 0
12 	 0
8 	 0
18 	 0
10 	 0
16 	 0
18 	 0
9 	 0
17 	 0
9 	 0
14 	 0
12 	 0
14 	 0
14 	 0
19 	 0
8 	 0
9 	 0
10 	 0


In [10]:
def isOrder(col):
    
    flag = 1
    aux = 0
    for i in col:
        if flag == 1:
            aux = i
            flag = 0
        else:
            if aux<=i:
                return False
    
    return True


def stress(layout):
    #Se revisa cada stack y se calcula el nivel de importancia que tiene para el coso
    st= []
    
    for i in layout.stacks:
        col = copy.deepcopy(i)
        col.reverse()
        #Ahora se obtiene el contador
        cont = 0
        for j in range(len(col)):
            cont += np.sqrt(col.pop(0)*(j+1))
            #print (cont)
        #print()   
        #Ahora se agrega a st
        st.append(cont)
    return st


In [11]:
def showtime(path):
    layout=px.read(path)
    
    
    #print ("Estado inicial: ")
    #for s in layout.stacks:
    #    print(s)
    #print()
    Greedy.lazy_greedy(layout)
    seleccionada = SBS(layout)
 
    while seleccionada != -1:
        
        if verbose:
            for s in layout.stacks: print(s)
            print("\n")
        
        #Se calcula cuantos contenedores deben retirarse
        columna=copy.deepcopy(layout.stacks[seleccionada])
        nb_unsorted_elements = px.remove_until(columna)

        #Se calculan los ranks
        rank = Greedy.get_ranks(layout.stacks[seleccionada])
        if verbose: print("selected:",layout.stacks[seleccionada])

        #Se retiran los elementos de la columna
        SDpp(layout, seleccionada, rank, nb_unsorted_elements)
        
        if verbose:
            for s in layout.stacks: print(s)
            print("\n")

        #Se quitan los contenedores que se mantuvieron en su lugar para que no tire error la siguiente funcion
        for item in layout.stacks[seleccionada]:
            rank.pop(item, None)

        #Según yo el último parámetro implica que el método tratará de llenar el stack, sin embargo si pongo 100 se cae?
        SFpp(layout,seleccionada,rank, layout.H-len(layout.stacks[seleccionada]) )#nb_unsorted_elements)

        Greedy.lazy_greedy(layout)
        #Se vuelve a seleccionar una columna
        seleccionada = SBS(layout)

    #print("SELECCIONADA:", seleccionada)
    #print(len(layout.moves),"\t",layout.unsorted_stacks)
    if verbose: 
        for s in layout.stacks: print(s)
    return len(layout.moves)
    #print()
    
    
def solve(path):
        layout=px.read(path)
    
    
        #print ("Estado inicial: ")
        #for s in layout.stacks:
        #    print(s)
        #print()
        Greedy.lazy_greedy(layout)
        seleccionada = SBS(layout)

        while seleccionada != -1:

            #Se calcula cuantos contenedores deben retirarse
            columna=copy.deepcopy(layout.stacks[seleccionada])
            nb_unsorted_elements = px.remove_until(columna)

            #Se calculan los ranks
            rank = Greedy.get_ranks(layout.stacks[seleccionada])
            #if verbose: print("selected:",layout.stacks[seleccionada])

            #Se retiran los elementos de la columna
            SDpp(layout, seleccionada, rank, nb_unsorted_elements)

            #if verbose:
            #   for s in layout.stacks: print(s)
            #   print("\n")

            #Se quitan los contenedores que se mantuvieron en su lugar para que no tire error la siguiente funcion
            for item in layout.stacks[seleccionada]:
                rank.pop(item, None)

            #Según yo el último parámetro implica que el método tratará de llenar el stack, sin embargo si pongo 100 se cae?
            SFpp(layout,seleccionada,rank, layout.H-len(layout.stacks[seleccionada]) )#nb_unsorted_elements)

            Greedy.lazy_greedy(layout)
            #Se vuelve a seleccionar una columna
            seleccionada = SBS(layout)

        #print("SELECCIONADA:", seleccionada)
        print(len(layout.moves),"\t",layout.unsorted_stacks)

        #print()

In [15]:
#SELECT Instancia
i=2
filename = "Instancias/"+ f+ "/data"+f+"-"+str(i)+".dat"

#Lectura del coso
layout = px.read(filename)

#print("Stacks")
#for i in layout.stacks:
#    print (i)
#print("\n")

#stress(layout)

def SBS(layout):
    
    st = stress(layout)
    #Pasemoslo a un array
    st = np.array(st)
    #print(st)
    
    #Recorro hasta ver un desordenado
    selected = -1
    best = 10000000
    
    for i in range(len(layout.stacks)):
        
        if layout.is_sorted_stack(i) == False:
            if st[i]<best:
                best=st[i]
                selected = i
    
    return selected    

    
print()
print()
verbose = False




In [28]:
f='10-10'
print("MINE")
for i in range(1,41):
    filename = "Instancias/"+ f+ "/data"+f+"-"+str(i)+".dat"
    solve(filename)
    
print()
print()
print("PROFE")


for i in range(1,41):
    filename = "Instancias/"+ f+ "/data"+f+"-"+str(i)+".dat"
    solve2(filename)

MINE
169 	 0
159 	 0
164 	 0
158 	 0
159 	 0
157 	 0
162 	 0
159 	 0
162 	 0
154 	 0
162 	 0
160 	 0
164 	 0
146 	 0
149 	 0
162 	 0
165 	 0
163 	 0
154 	 0
160 	 0
164 	 0
162 	 0
159 	 0
172 	 0
163 	 0
176 	 0
166 	 0
156 	 0
170 	 0
165 	 0
164 	 0
148 	 0
162 	 0
169 	 0
158 	 0
162 	 0
156 	 0
160 	 0
162 	 0
179 	 0


PROFE
163 	 0
159 	 0
164 	 0
170 	 0
159 	 0
157 	 0
160 	 0
159 	 0
165 	 0
154 	 0
165 	 0
163 	 0
160 	 0
146 	 0
154 	 0
162 	 0
163 	 0
164 	 0
154 	 0
161 	 0
160 	 0
158 	 0
151 	 0
172 	 0
162 	 0
163 	 0
166 	 0
156 	 0
169 	 0
160 	 0
155 	 0
148 	 0
164 	 0
171 	 0
161 	 0
162 	 0
157 	 0
162 	 0
165 	 0
175 	 0
